# Coursera

https://www.edarabia.com/schools/sweden/

scrape this for school list

In [ ]:
# get drivers ready for selenium scraping
from zipfile import ZipFile
import wget

wget.download('https://github.com/mozilla/geckodriver/releases/download/v0.27.0/geckodriver-v0.27.0-win64.zip', 'gecko.zip')
with ZipFile('gecko.zip', 'r') as z: z.extractall('.')

In [16]:
# scrape https://www.edarabia.com/schools/sweden/ for top schools of sweden
from selenium import webdriver

driver = webdriver.Firefox(executable_path='geckodriver.exe')
driver.get('https://www.edarabia.com/schools/sweden/')

In [12]:
# Upon inspection, school names are part of a hyperlink under <h5> tags
links = driver.find_elements_by_xpath('//h5/a')
for i, link in enumerate(links):
    print(i, link.text)

0 
1 
2 
3 
4 Bladins International School
5 Deutsche Schule Stockholm
6 Engelska Skolan Norr AB
7 Fryshuset School
8 International IT College of Sweden
9 Lycee Francais Saint Louis in Stockholm
10 ProCivitas Private Gymnasium Stockholm
11 Swedish Finnish School
12 The Tanto International School
13 Thea Private Grundskola


In [14]:
# there seem to be empty links, lets remove that
schools = [link.text for link in links if link.text]
schools

['Bladins International School',
 'Deutsche Schule Stockholm',
 'Engelska Skolan Norr AB',
 'Fryshuset School',
 'International IT College of Sweden',
 'Lycee Francais Saint Louis in Stockholm',
 'ProCivitas Private Gymnasium Stockholm',
 'Swedish Finnish School',
 'The Tanto International School',
 'Thea Private Grundskola']

In [17]:
# let's see if we can get their address.
address = driver.find_elements_by_xpath('//ul/li')
for i, addr in enumerate(address):
    print(i, addr.text)

0 
1 Universities
2 Schools
3 Nurseries
4 Courses
5 Jobs
6 Events
7 Login
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 Home
56 Schools
57 Sweden
58 Preschool (förskola)
59 Compulsory schooling which is made up of Elementary school (lågstadiet), middle school (mellanstadiet) and junior high school (högstadiet).
60 Upper secondary school or senior high school (gymnasium)
61 Tertiary education
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 Address: Sjallandstorget 1
76 Curriculum: IB
77 SEK 23,500
78 Address: Karlavagen 25
79 Address: Roslagstullsbacken 4
80 Founded: 1993
81 Address: Martensdalsgatan 2-8
82 Founded: 1984
83 Address: Halsobrunnsgatan 6
84 Founded: 2003
85 Address: Essingestraket 24
86 Founded: 1959
87 Curriculum: French
88 SEK 12,375
89 Address: Sandbacksgatan 10
90 Address: Ryttargatan 275, Planner
91 Address: Flintbacken 

In [25]:
# we want one that starts with "Address:"
school_addresses = []
for addr in address:
    addr = addr.text
    if "Address:" in addr:
        addr.replace('Address: ', '')
        # the , Planner in 1 of the address is causing problem with geolocation
        addr = addr.split(',')[0]
        school_addresses.append(addr.replace('Address: ', ''))
school_addresses


['Sjallandstorget 1',
 'Karlavagen 25',
 'Roslagstullsbacken 4',
 'Martensdalsgatan 2-8',
 'Halsobrunnsgatan 6',
 'Essingestraket 24',
 'Sandbacksgatan 10',
 'Ryttargatan 275',
 'Flintbacken 20',
 'Tullgardsgatan 10']

In [26]:
# check if we get same number of addresses and schools
print(len(schools))
print(len(school_addresses))

10
10


In [28]:
# see if we can get coordinates from address
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='my_agent')
lat = []
long = []
for addr in school_addresses:
    loc = geolocator.geocode(f'{addr}, Sweden')
    lat.append(loc.latitude)
    long.append(loc.longitude)


In [31]:
# check leng of lat and long
print(len(lat))
print(len(long))

10
10


In [33]:
# all the list seem correct. let's make this into a dataframe
import pandas as pd

df = pd.DataFrame(zip(schools, school_addresses, lat, long), columns=['School','Address','Lat','Long'])
df

,School,Address,Lat,Long
0,Bladins International School,Sjallandstorget 1,55.592730,12.987777
1,Deutsche Schule Stockholm,Karlavagen 25,59.345046,18.063079
2,Engelska Skolan Norr AB,Roslagstullsbacken 4,59.350678,18.059369
3,Fryshuset School,Martensdalsgatan 2-8,59.300544,18.088212
4,International IT College of Sweden,Halsobrunnsgatan 6,59.337121,18.047313
5,Lycee Francais Saint Louis in Stockholm,Essingestraket 24,59.319225,17.989403
6,ProCivitas Private Gymnasium Stockholm,Sandbacksgatan 10,59.316701,18.080628
7,Swedish Finnish School,Ryttargatan 275,59.507273,17.907965
8,The Tanto International School,Flintbacken 20,59.309693,18.049634
9,Thea Private Grundskola,Tullgardsgatan 10,59.305603,18.084816


In [ ]:
# looking perfect. Let's save this into csv so we can continue working without having to re-scrape
df.to_csv('schools.csv', index=False)